# Can't figure out how to import Tslearn



In [1]:

import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import seaborn as sns
import statistics
#import sklearn
import tslearn
import math 
import os
import scipy.stats as stats
import glob
import warnings

from statistics import mode
from scipy import signal
from scipy.signal import find_peaks, peak_prominences

from numpy.lib.stride_tricks import sliding_window_view

from tslearn.clustering import TimeSeriesKMeans
from tslearn.metrics import dtw
from tslearn.utils import to_time_series_dataset

/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


# Import, Reorder & Label CSVs

In [ ]:
path = '/Users/juliapersche/Desktop/darpa_datascience folder/A_DeviceMotion_data/User_1'
csv_files = glob.glob(os.path.join(path, "*.csv"))
print(csv_files)

In [ ]:
indices = [0,2,4,5,8,3,6,10,9,11,1,7,12,13,14]
data = [csv_files[i] for i in indices]
print(data)

In [ ]:
list_of_dfs = []
activities = ['jog', 'jog','dws','dws','dws', 'wlk','wlk', 'wlk','std','std','ups','ups','ups','sit','sit']
trials = [16,9,11,1,2,8,15,7,14,6,12,4,3,5,13]

counter = 0
for f in data:
       # read the csv file
        k = pd.read_csv(f).rename(columns={'Unnamed: 0': 'time'})
        #Label
        k['activity'] = counter
        k['trial'] = trials[counter]
        list_of_dfs.append(k)
        counter += 1
        

df = pd.concat(list_of_dfs).set_index('time')
df = df.drop(['attitude.roll', 'attitude.pitch', 'attitude.yaw', 'gravity.x', 'gravity.y', 'gravity.z'], axis=1)
df = df.rename(columns={'rotationRate.x': 'gyro_x', 'rotationRate.y': 'gyro_y', 'rotationRate.z': 'gyro_z'})

# Compute Accel Energy & Resample Data

In [3]:
df['x_norm'] = (df['userAcceleration.x'] - df['userAcceleration.x'].mean()) / (df['userAcceleration.x'].max() - df['userAcceleration.x'].min())
df['y_norm'] = (df['userAcceleration.y'] - df['userAcceleration.y'].mean()) / (df['userAcceleration.y'].max() - df['userAcceleration.y'].min())
df['z_norm'] = (df['userAcceleration.z'] - df['userAcceleration.z'].mean()) / (df['userAcceleration.z'].max() - df['userAcceleration.z'].min())

df['energy'] = (abs(df['x_norm']**2 + df['y_norm']**2 + df['z_norm']**2))**.5

df.head()

NameError: name 'df' is not defined

In [ ]:
#resampling (from 50 to 20 Hz) --> unsure why resampled from 50 to 20 Hz
df['time'] = np.arange(0, len(df)*20, 20)
df['time'] = pd.to_timedelta(df['time'], unit='ms')
df = df.set_index('time')
df_resample = df.resample('50ms').mean()
df['time'] = df_resample.set_index
df

# Visualize Energy of Different Activities

In [ ]:
fig, axs = plt.subplots(6, 1, sharex ='col')

axs[0].plot(x, df.loc[df['activity'] == 1]['energy'][0:200])
axs[0].set_ylabel('downstairs \n energy')
axs[1].plot(x, df.loc[df['activity'] == 0]['energy'][0:200])
axs[1].set_ylabel('jog \n energy')
axs[2].plot(x, df.loc[df['activity'] == 5]['energy'][0:200])
axs[2].set_ylabel('sit \n energy')
axs[3].plot(x, df.loc[df['activity'] == 3]['energy'][0:200])
axs[3].set_ylabel('standing \n energy')
axs[4].plot(x, df.loc[df['activity'] == 4]['energy'][0:200])
axs[4].set_ylabel('upstairs \n energy')
axs[5].plot(x, df.loc[df['activity'] == 2]['energy'][0:200])
axs[5].set_ylabel('walking \n energy')

In [ ]:
#Splitting energy computed for each trial into 10 second windos with 5 second overlap
energy_windows = sliding_window_view(stats.zscore((df['energy']).to_numpy()), 200)[::150, :]

#Splitting up corresponding labels
activity_windows = sliding_window_view((df['activity']), 200)[::150, :]

energy_windows.shape
print(energy_windows)

# Create Dictionary with keys for each activity and corresponding windows 
### This dictionary will serve as 'ground truth' when determining the accuracy of the kmeans clustering assignment for each unlabeled energy window.

In [ ]:
activity_labels = [statistics.mode(x) for x in activity_windows]
activity_splits = {}
for i, act in enumerate(activity_labels):
    activity = activities[act]
    if activity_splits.get(activity) == None:
        activity_splits[activity] = []
    activity_splits[activity].append(i)

print(activity_splits)

#Printing number of activities in each key 
for key, value in activity_splits.items():
    #print value
    print(key, len([item for item in value if item]))

### Remap assigned labels to actual activity labels 

In [ ]:
#jog 0 --> 1 
#dws 1 --> 0 
#wlk 2 --> 2
#std 3 --> 5
#ups 4 --> 0 
#sit 5 --> 5 

for key, value in activity_splits.items():
    if key == 'jog':
          jog_label = np.full(len([item for item in value if item])+1, 1, dtype=int)
    if key == 'dws':
           dws_label = np.full(len([item for item in value if item]), 0, dtype=int)
    if key == 'wlk':
         wlk_label = np.full(len([item for item in value if item]), 2, dtype=int)
    if key == 'std':
         std_label = np.full(len([item for item in value if item]), 5, dtype=int)
    if key == 'ups':
         ups_label = np.full(len([item for item in value if item]), 0, dtype=int)
    if key == 'sit':
         sit_label = np.full(len([item for item in value if item]), 5, dtype=int)
    
labels = np.concatenate((jog_label,dws_label,wlk_label,std_label,ups_label,sit_label))
labels.shape
print(len(labels))

print(np.sum(labels==0))
print(labels)

# Createte Kmeans model with unlabeled Energy Windows
### Since we are working with Time Series Data, we expect to see a shift in the windows vs the cluster center segments.  To account for this, use Dynamic Time Warping

In [ ]:
#Energy
k = 6
E = energy_windows
km_e = TimeSeriesKMeans(n_clusters=k, metric="softdtw", max_iter=50,
                         max_iter_barycenter=5,
                         random_state=0).fit(E)
#print(km_e.cluster_centers_)

# Feed energy windows into kmeans model.  

In [ ]:
predict = km_e.predict(E)
print(predict)

In [ ]:
#see which are correct
scores = (labels == predict)
#combining labeled and predicted cluster scores
actual_predicted = np.hstack([labels[:, np.newaxis], predict[:, np.newaxis],scores[:, np.newaxis]])
# #print(actual_predicted[0][2])

### Compute overall model accuracy

In [ ]:
#actual_predicted_ts[i][2]
model_accuracy = (sum(scores)/len(scores)) *100
print(model_accuracy)

### Group model prediction for each cluster per activity

In [ ]:
activities_list = ['jog', 'dws', 'wlk','std','ups','sit']

print("AX cluster scores")
for act in activities_list:
    print(act)
    predictE = km_e.predict(E[activity_splits[act]])
    print(predictE)
    print('-----') 

In [ ]:
for act in activities_list:
     predictE = km_e.predict(E[activity_splits[act]])
     if act == 'jog':
          jog_predict = np.array(predictE)
     if act == 'dws':
         dws_predict = np.array(predictE)
     if act == 'wlk':
         wlk_predict = np.array(predictE)
     if act == 'std':
         std_predict = np.array(predictE)
     if act == 'ups':
        ups_predict = np.array(predictE)
     if act == 'sit':
        sit_predict = np.array(predictE)
        
print(len(jog_predict))     
print(len(jog_label))  

# Visualize Prediction Accuracy

In [ ]:
window = np.arange(0,415,1)
activityline_dict = {0:'jog begin', 1:'dws begin', 2: 'walk begin', 3:'std begin', 4: 'ups begin', 5: 'sit begin'}
activity_colorlabel_dict = {0:'orange', 1:'b', 2: 'k', 3:'c', 4: 'y', 5: 'm'}
activitylabel_dict = {0:'dws', 1:'jog', 2: 'wlk ', 3:'N/A', 4: 'walk ', 5: 'std / sit '}


activitystart_dict = {0:0, 1: 43, 2: 77, 3:151, 4: 236, 5: 278}
k = 6

scores_list = scores.tolist()
scores_list_len = len(scores_list)
score_colors = []

for j in range(scores_list_len):
    if scores[j] == 0:
      score_colors+= ["red"]
    if scores[j] == 1:
         score_colors+= ["green"]

score_colors = np.asarray(score_colors)
score_colors_dict = dict(zip(window, score_colors))

In [ ]:
for i in range(k):
   plt.axvline(x = activitystart_dict[i], color = 'b', label = activityline_dict[i])

for j in range(scores_list_len):
   plt.scatter(window[j],scores[j],color = score_colors_dict[j],alpha = .5)

plt.rcParams["figure.figsize"] = [30,6]
plt.legend()
plt.title('Label Prediction Accuracy')
plt.show()

# Visualize actual vs predicted activity for each windo

In [ ]:
predictedlabels_colors = []
actuallabels_colors = []
for j in range(scores_list_len):
    if labels[j] == 0:
      actuallabels_colors+= ['orange']
    if labels[j] == 1:
        actuallabels_colors+= ['b']
    if labels[j] == 2:
        actuallabels_colors+= ['k']
    if labels[j] == 3:
         actuallabels_colors+= ['c']
    if labels[j] == 4:
         actuallabels_colors+= ['y']
    if labels[j] == 5:
         actuallabels_colors+= ['m']

    for r in range(scores_list_len):
        if predict[r] == 0:
            predictedlabels_colors += ['orange']
        if predict[r] == 1:
            predictedlabels_colors += ['b']
        if predict[r] == 2:
            predictedlabels_colors += ['k']
        if predict[r] == 3:
            predictedlabels_colors += ['c']
        if predict[r] == 4:
            predictedlabels_colors += ['y']
        if predict[r] == 5:
            predictedlabels_colors += ['m']


actuallabels_colors = np.asarray(actuallabels_colors)
actuallabels_colors_dict = dict(zip(window, actuallabels_colors))

predictedlabels_colors = np.asarray( predictedlabels_colors)
predictedlabels_colors_dict = dict(zip(window,  predictedlabels_colors))
print(predictedlabels_colors_dict)

In [ ]:
fig, axs = plt.subplots(2, 1, sharex ='col')

for i in range(k):
   axs[0].axvline(x = activitystart_dict[i], color = 'b')
   axs[0].scatter(x = 0,y=0, color = activity_colorlabel_dict[i], label = activitylabel_dict[i])
   axs[0].legend()

for j in range(scores_list_len):
    axs[0].scatter(window[j],labels[j],color = actuallabels_colors_dict[j],alpha = .5)

axs[0].set_ylabel('Actual \n Labels')
axs[0].set_xlabel('Window')


for i in range(k):
   axs[1].axvline(x = activitystart_dict[i], color = 'b', label = activityline_dict[i])
   axs[1].legend()

for j in range(scores_list_len):
    axs[1].scatter(window[j],predict[j],color = predictedlabels_colors_dict[j],alpha = .5)
axs[1].set_ylabel('Predicted \n Labels')
axs[1].set_xlabel('Window')

# Compute accuracy per activity & visualize in a confusion matrix

In [ ]:
print('jog predict mode:', stats.mode(jog_predict), (stats.mode(jog_predict)[1][0]/len(jog_predict)*100))
print('dws predict mode:', stats.mode(dws_predict), (stats.mode(dws_predict)[1][0]/len(dws_predict))*100)
print('wlk predict mode:', stats.mode(wlk_predict), (stats.mode(wlk_predict)[1][0]/len(wlk_predict))*100)
print('std predict mode:', stats.mode(std_predict), (stats.mode(std_predict)[1][0]/len(std_predict))*100)
print('ups predict mode:', stats.mode(ups_predict), (stats.mode(ups_predict)[1][0]/len(ups_predict))*100)
print('sit predict mode:', stats.mode(sit_predict), (stats.mode(sit_predict)[1][0]/len(sit_predict))*100)